# Interacting with Coq

The `Pytanque` class implements a lightweight client that can communicate with the Rocq prover via the `pet-server`.

In [12]:
from pytanque import Pytanque

First you need to start the server.
Instructions can be found in the coq-lsp repo (https://github.com/ejgallego/coq-lsp/tree/main/petanque)

Then, you can create a client and connect it to the same address and port.
You need to indicate the location of the project (directory containing the `_CoqProject` file, or root directory for the `.v` files).

In [13]:
pet = Pytanque("127.0.0.1", 8080)
pet.connect()
env = pet.init(root=".")

Petanque relies on regular coq files.
To prove an existing theorem, just indicate the location of the `.v` file and the name of the theorem.

In [14]:
with open("./scratch.v", "w") as file:
    code = """
Lemma test n : 1 + n > n.
    """
    print(code, file=file)


state = pet.start(file="./scratch.v", thm="test")

PetanqueError: Error(code=-32005, message='Theorem_not_found: [find_thm] Theorem not found!')

Internally the state is a simple `id`.
To retrieve the state of the prover, you can use the `goals` method.

In [ ]:
print(f"Internal pytanque state: {state}")

goals = pet.goals(state)  # Retrieve actual state
print(f"Prover goals: {goals}\n")
print(goals.pp())  # Pretty print the current goals

Internal pytanque state: CurrentState(value=1)
Prover goals: GoalsResponse(goals=[Goal(info={'evar': ['Ser_Evar', 3], 'name': None}, hyps=[GoalHyp(names=['n'], ty='nat', def_=None)], ty='1 + n > n')])

n  : nat
-----------------------------
1 + n > n


To execute a tactic, use the `run_tac` method.

In [ ]:
state = pet.run_tac(state, "induction n.")
goals = pet.goals(state)
print(goals.pp())


-----------------------------
1 + 0 > 0

n  : nat
IHn  : 1 + n > n
-----------------------------
1 + S n > S n


If the proof is finished (no more goal) the state becomes `ProofFinished`

In [ ]:
state = pet.run_tac(state, "auto.")
state = pet.run_tac(state, "auto.")
state

ProofFinished(value=4)

# Proof search

Adapted from the NeurIPS 2024 tutorial: https://github.com/yangky11/ml4tp-tutorial/blob/main/main.ipynb 

Let us start with a dummy tactic generator (random sampling in a list of tactics) with random scores for each tactics in $[0, 1]$.

In [ ]:
import random
import pytanque as pt
from typing import List, Tuple, Optional

Tactic = str
Proof = List[Tactic]
Score = float


def generate_tactics(goal: str, num_candidates: int) -> Tuple[List[str], List[float]]:
    tactics = [
        "intros.",
        "auto.",
        "simpl.",
        "induction n.",
        "easy.",
        "trivial.",
        "rewrite IHn.",
    ]
    return random.choices(tactics, k=num_candidates), [
        random.uniform(0, 1) for _ in range(num_candidates)
    ]

We can now implement depth-first search to search for proofs.

In [ ]:
num_candidates = 16
depth_limit = 3


def depth_first_search(state: pt.State, depth: int) -> Optional[Proof]:
    """Try to prove `state` using depth-first search (DFS)."""
    if depth >= depth_limit:
        return None

    tactics, _ = generate_tactics(state, num_candidates)

    # Run the tactics.
    for tac in tactics:
        try:
            next_state = pet.run_tac(state, tac)
            match next_state:
                case pt.ProofFinished(st):
                    return [tac]
                case pt.CurrentState(st):
                    subproof = depth_first_search(next_state, depth + 1)
                    if subproof:
                        return [tac] + subproof
        except pt.PetanqueError:
            pass
    return None

Let us try on a simple example.

In [ ]:
state = pet.start(file="./scratch.v", thm="test")
proof = depth_first_search(state, depth=0)

if proof:
    print(f"Proof: {" ".join(proof)}")
else:
    print("Not proof found...")

Proof: trivial. trivial. auto.


Here is an implementation of best-first search adapted from the 2023 Neural theorem proving tutorial https://github.com/wellecks/ntptutorial

In [ ]:
import heapq
from tqdm import trange
import logging


def best_first_search(
    init_state: pt.State, max_iters: int, num_samples: int
) -> Optional[Proof]:
    queue = [(0.0, [], init_state)]
    visited = set()
    for _ in trange(max_iters):
        if len(queue) == 0:
            break

        total_score, steps, state = heapq.heappop(queue)
        goals = pet.goals(state)
        visited.add(goals.pp())

        step_cands, step_scores = generate_tactics(state, num_samples)

        for step, score in zip(step_cands, step_scores):
            try:
                result = pet.run_tac(state, step)
                match result:
                    case pt.ProofFinished():
                        return steps + [step]
                    case pt.CurrentState():
                        next_goals = pet.goals(result)
                        if next_goals.pp() not in visited:
                            # Score is negative log probability summed across steps
                            new_score = total_score - score
                            heapq.heappush(queue, (new_score, steps + [step], result))
            except pt.PetanqueError as err:
                logging.warning(f"Invalid Tactic: {step} {err}")
    return None

In [ ]:
state = pet.start(file="./scratch.v", thm="test")
proof = best_first_search(state, max_iters=20, num_samples=5)

if proof:
    print(f"Proof: {" ".join(proof)}")
else:
    print("Not proof found...")

  0%|          | 0/20 [00:00<?, ?it/s]

Proof: auto.
